In [1]:
import openmm
import openmm.app
import openmm.unit

from collections import defaultdict

In [2]:
prot = openmm.app.PDBFile('apo.pdb')

In [3]:
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml','phenol_openff.xml')

In [4]:
prot_system = ff.createSystem(prot.topology)
integrator = openmm.VerletIntegrator(
    1.0 * openmm.unit.femtosecond
)

In [6]:
for f in range(prot_system.getNumForces()):
    force = prot_system.getForce(f)
    force.setForceGroup(f)
    print(force.__class__.__name__, force.getForceGroup())
    if force.__class__.__name__ == 'NonbondedForce':
        nbforce = force

HarmonicBondForce 0
NonbondedForce 1
PeriodicTorsionForce 2
CMMotionRemover 3
HarmonicAngleForce 4


In [10]:
print(nbforce.getNumExceptions())

14357


In [11]:
for i in range(nbforce.getNumExceptions()):
    print(nbforce.getExceptionParameters(i))

[0, 1, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[0, 2, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[1, 2, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[0, 3, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[1, 3, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[2, 3, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[0, 4, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[1, 4, Quantity(value=0.0, unit=elementary charge**2), Quantit

In [6]:
simulation = openmm.app.Simulation(prot.topology,prot_system,integrator)
simulation.context.setPositions(prot.positions)

In [7]:
for f in range(prot_system.getNumForces()):
    print(prot_system.getForce(f).__class__.__name__)
    print(simulation.context.getState(getEnergy=True,groups={f}).getPotentialEnergy())

HarmonicBondForce
36422.11169433594 kJ/mol
NonbondedForce
-22913.933364868164 kJ/mol
PeriodicTorsionForce
14300.93374633789 kJ/mol
CMMotionRemover
0.0 kJ/mol
HarmonicAngleForce
17081.651977539062 kJ/mol


## With ghost molecules

In [8]:
pdb = openmm.app.PDBFile('gcmc-ghosts.pdb')

In [9]:
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml','phenol_openff.xml')

In [10]:
pdb_system = ff.createSystem(pdb.topology)
integrator = openmm.VerletIntegrator(
    1.0 * openmm.unit.femtosecond
)

In [11]:
for f in range(pdb_system.getNumForces()):
    force = pdb_system.getForce(f)
    force.setForceGroup(f)
    print(force.__class__.__name__, force.getForceGroup())

HarmonicBondForce 0
NonbondedForce 1
PeriodicTorsionForce 2
CMMotionRemover 3
HarmonicAngleForce 4


In [12]:
simulation = openmm.app.Simulation(pdb.topology,pdb_system,integrator)
simulation.context.setPositions(pdb.positions)

In [13]:
frag_res_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)

In [14]:
for f in range(pdb_system.getNumForces()):
    force = pdb_system.getForce(f)
    if force.__class__.__name__ == "NonbondedForce":
        nonbonded_force = force

In [15]:
frag_atom_ids = defaultdict(list)
for resid, residue in enumerate(pdb.topology.residues()):
    if resid in frag_res_ids:
        for atom in residue.atoms():
            frag_atom_ids[resid].append(atom.index)
            nonbonded_force.setParticleParameters(atom.index, 0., 0., 0.,)
nonbonded_force.updateParametersInContext(simulation.context)

In [16]:
for f in range(pdb_system.getNumForces()):
    print(pdb_system.getForce(f).__class__.__name__)
    print(simulation.context.getState(getEnergy=True,groups={f}).getPotentialEnergy())

HarmonicBondForce
36425.608642578125 kJ/mol
NonbondedForce
-23327.65020751953 kJ/mol
PeriodicTorsionForce
14300.936645507812 kJ/mol
CMMotionRemover
0.0 kJ/mol
HarmonicAngleForce
17402.5439453125 kJ/mol
